### Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import TruncatedSVD
import torch
from transformers import AutoModel, AutoTokenizer

/Users/svlg/opt/anaconda3/envs/ML4H/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [57]:
# loading data with preprocessed tweets
path = 'data/'
# data = pd.read_csv(path + 'tweets_only.csv')
# train = pd.read_csv(path + 'train_tweets_only.csv')
# test = pd.read_csv(path + 'test_tweets_only.csv')
# val = pd.read_csv(path + 'val_tweets_only.csv')
log_reg_class = pd.read_csv(path + 'LogReg.csv')
knn_class = pd.read_csv(path + 'KNN.csv')
rf_class = pd.read_csv(path + 'RF.csv')


In [58]:
log_reg_class

,Methode,BagOfWords,TF-IDF,FastText_Mean,FastText_Max,FastText_PCA,Word2Vec_Mean,Word2Vec_Max,Word2Vec_PCA,Glove_Mean,Glove_Max,Glove_PCA
0,Logistic Regression Positive,0.8054,0.8002,0.4914,0.4816,0.4363,0.6181,0.5963,0.6248,0.5640,0.5070,0.5475
1,Logistic Regression Negative,0.7417,0.7368,0.5389,0.5380,0.5392,0.6274,0.5831,0.6368,0.5663,0.5423,0.5476


In [59]:
knn_class

,Methode,BagOfWords,TF-IDF,FastText_Mean,FastText_Max,FastText_PCA,Word2Vec_Mean,Word2Vec_Max,Word2Vec_PCA,Glove_Mean,Glove_Max,Glove_PCA
0,Logistic Regression Positive,0.4061,0.4017,0.5191,0.4695,0.4758,0.5546,0.5276,0.5632,0.5475,0.5036,0.5507
1,Logistic Regression Negative,0.5369,0.5371,0.5469,0.5368,0.5386,0.6010,0.5603,0.5953,0.5792,0.5417,0.5670


In [60]:
rf_class

,Methode,BagOfWords,TF-IDF,FastText_Mean,FastText_Max,FastText_PCA,Word2Vec_Mean,Word2Vec_Max,Word2Vec_PCA,Glove_Mean,Glove_Max,Glove_PCA
0,Random Forrest Classifer Positive,0.7979,0.8012,0.5011,0.4890,0.4775,0.5166,0.4761,0.5225,0.5379,0.5235,0.5383
1,Random Forrest Classifer Negative,0.7414,0.7378,0.5390,0.5453,0.5411,0.5411,0.5366,0.5402,0.5450,0.5390,0.5417


In [61]:
cols = [['Methode','BagOfWords','TF-IDF','FastText','FastText','FastText','Word2Vec','Word2Vec','Word2Vec','Glove','Glove','Glove'],
        ['Embedding','Tweetwise','Tweetwise','Mean','Max','PCA','Mean','Max','PCA','Mean','Max','PCA']]

embedding_class = pd.concat((log_reg_class,knn_class,rf_class))
embedding_class.columns = cols
embedding_class

Methode BagOfWords    TF-IDF FastText           
                           Embedding  Tweetwise Tweetwise     Mean     Max   
0       Logistic Regression Positive     0.8054    0.8002   0.4914  0.4816  \
1       Logistic Regression Negative     0.7417    0.7368   0.5389  0.5380   
0       Logistic Regression Positive     0.4061    0.4017   0.5191  0.4695   
1       Logistic Regression Negative     0.5369    0.5371   0.5469  0.5368   
0  Random Forrest Classifer Positive     0.7979    0.8012   0.5011  0.4890   
1  Random Forrest Classifer Negative     0.7414    0.7378   0.5390  0.5453   

          Word2Vec                   Glove                  
      PCA     Mean     Max     PCA    Mean     Max     PCA  
0  0.4363   0.6181  0.5963  0.6248  0.5640  0.5070  0.5475  
1  0.5392   0.6274  0.5831  0.6368  0.5663  0.5423  0.5476  
0  0.4758   0.5546  0.5276  0.5632  0.5475  0.5036  0.5507  
1  0.5386   0.6010  0.5603  0.5953  0.5792  0.5417  0.5670  
0  0.4775   0.5166  0.4761  0.5225  0.5379  0.5235  0.5383  
1  0.5411   0.5411  0.5366  0.5402  0.5450  0.5390  0.5417

In [65]:
embedding_class['Target'] = ['Positive','Negative','Positive','Negative','Positive','Negative']
embedding_class['Classifier']  = ['Logistic Regression','Logistic Regression','KNeighbors','KNeighbors','Random Forest','Random Forest']

In [69]:
embedding_class = embedding_class.set_index(['Classifier','Target'])

In [72]:
embedding_class = embedding_class.drop(columns=['Methode'])

/var/folders/n6/w16z2dmj2g9d1pd2dt_r8xl80000gn/T/ipykernel_27251/2832346039.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  embedding_class = embedding_class.drop(columns=['Methode'])


In [73]:
embedding_class

BagOfWords    TF-IDF FastText                   
                              Tweetwise Tweetwise     Mean     Max     PCA   
Classifier          Target                                                   
Logistic Regression Positive     0.8054    0.8002   0.4914  0.4816  0.4363  \
                    Negative     0.7417    0.7368   0.5389  0.5380  0.5392   
KNeighbors          Positive     0.4061    0.4017   0.5191  0.4695  0.4758   
                    Negative     0.5369    0.5371   0.5469  0.5368  0.5386   
Random Forest       Positive     0.7979    0.8012   0.5011  0.4890  0.4775   
                    Negative     0.7414    0.7378   0.5390  0.5453  0.5411   

                             Word2Vec                   Glove                  
                                 Mean     Max     PCA    Mean     Max     PCA  
Classifier          Target                                                     
Logistic Regression Positive   0.6181  0.5963  0.6248  0.5640  0.5070  0.5475  
                    Negative   0.6274  0.5831  0.6368  0.5663  0.5423  0.5476  
KNeighbors          Positive   0.5546  0.5276  0.5632  0.5475  0.5036  0.5507  
                    Negative   0.6010  0.5603  0.5953  0.5792  0.5417  0.5670  
Random Forest       Positive   0.5166  0.4761  0.5225  0.5379  0.5235  0.5383  
                    Negative   0.5411  0.5366  0.5402  0.5450  0.5390  0.5417

In [ ]:
w2v = AutoModel.from_pretrained('vocab-transformers/distilbert-word2vec_256k-MLM_best')
w2v_token = AutoTokenizer.from_pretrained('vocab-transformers/distilbert-word2vec_256k-MLM_best')

In [ ]:
def get_PC1(X):
    pca = TruncatedSVD(n_components=1)
    pca.fit(X)
    return pca.components_


def get_w2v_embeddings(df,w2v,w2v_token):
    df_out = []
    for t in tqdm(df['TweetText']):
        tokens = w2v_token.tokenize(t)
        token_ids = w2v_token.convert_tokens_to_ids(tokens)
        token_ids = torch.tensor([token_ids])
        
        with torch.no_grad():
            tweet_emb = w2v(token_ids)

        tweet_emb = tweet_emb.last_hidden_state.squeeze().numpy()
        tweet_mean = tweet_emb.mean(axis=0)
        tweet_max = np.amax(np.abs(tweet_emb),axis=0)
        if not len(tokens) == 1:
            tweet_pca = get_PC1(tweet_emb).squeeze()
        else:
            tweet_pca = tweet_emb
        
        df_out.append((tweet_mean,tweet_max,tweet_pca))

    df = pd.DataFrame(df_out, columns=['MeanEmbeddings','MaxEmbeddings','PCAEmbeddings'])
    return df

In [ ]:
n = 5
train_emb = get_w2v_embeddings(train.iloc[:n], w2v, w2v_token)
train = train.iloc[:n].join(train_emb).drop(columns=['TweetText'])

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
test = pd.read_csv('data/w2v_test_10k.csv')

In [55]:
test

,TweetId,TweetText,Positive,Negative
0,1.236277e+18,once again many thanks to the 546 spectator pl...,3,-1
1,1.229550e+18,houston along with charlotte and the la clippe...,1,-1
2,1.224769e+18,terrence 142300 per his financial statement wi...,1,-1
3,1.240085e+18,i appeal everyone coming from abroad to mandat...,1,-2
4,1.243814e+18,coronavirus president hope crisis lead society...,2,-1
...,...,...,...,...
56177,1.238171e+18,i will say to the lord “ my refuge and my fort...,3,-1
56178,1.242540e+18,prisoner released early due to the coronavirus...,3,-3
56179,1.223727e+18,lmaooo i mad this happen right on the corner f...,1,-2
56180,1.254461e+18,i just really cannot belive no one ha copped t...,2,-1


In [56]:
test = pd.read_csv(path + 'w2v_test_10k.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/w2v_test_10k.csv'